In [ ]:
pip install openseespy

## **Modeling this Turbine**
**65-kW Wind Turbine courtesy of UCSD**
<img src='Research\Turbine.PNG' width="300" height="300">

In [1]:
# following example on http://opensees.berkeley.edu/wiki/index.php/Elastic_Frame_Example
# running dynamic ground motion

%matplotlib notebook

# import OpenSees and libraries
from openseespy.opensees import *
from openseespy.postprocessing.Get_Rendering import *

# numerical and plotting tools
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import math
from math import sqrt
import pandas as pd
import csv

# system commands
import os, os.path
import glob
import shutil

In [6]:
def buildmodel():

    # -------------------------------
    #       Generate Model
    # -------------------------------

    # remove existing model
    wipe()

    # remove existing results
    # explanation here: https://stackoverflow.com/a/31989328
    def remove_thing(path):
        if os.path.isdir(path):
            shutil.rmtree(path)
        else:
            os.remove(path)

    def empty_directory(path):
        for i in glob.glob(os.path.join(path, '*')):
            remove_thing(i)

    empty_directory('modes')
    empty_directory('output')

    # ---------------------------------------
    #   Generate model and static analysis
    # ---------------------------------------

    # set modelbuilder
    model('basic', '-ndm', 3, '-ndf', 6)

    # units: in, kip, s
    # dimensions
    ft = 12.0
    inch = 1.0
    g = 386.1 #in/s^2
    kip = 1.0
    ksi = kip/(inch**2)
    s = 1

    # material properties
    Es = 29000*ksi
    Gs = 11500*ksi

    # -------------------------------
    #       Create Nodes
    # -------------------------------
    # command: node(nodeID, x-coord, y-coord, z-coord)
    # command: node(nodeTag, *crds, '-ndf', ndf, '-mass', *mass, '-disp', *disp, '-vel', *vel, '-accel', *accel)
    # Note: Ian Prowell Dissertation: Split into 30 beam-column elements

    # specify number of elements of each portion of the tower
    eldiv = 6

    # Defining the first node
    nodeTag = 1
    node(nodeTag, 0.0, 0.0, 0.0)

    # Heights of tower sections
    hbot = 238.2*inch
    hbottap = 74.8*inch
    hmid = 237.2*inch
    hmidtap = 74.8*inch
    htop = 263*inch # to reach the top of the nacelle 238 og 

    # Lower tower section (Split into eldiv elements)
    for j in range(eldiv):
        nodeTag += 1
        h = (hbot/eldiv)*(j + 1)
        node(nodeTag, 0.0, 0.0, h)

    # Tapered lower section (Split into eldiv elements)
    for j in range(eldiv):
        nodeTag += 1
        h = hbot + (hbottap/eldiv)*(j + 1)
        node(nodeTag, 0.0, 0.0, h)

    # Middle tower section (Split into eldiv elements)
    for j in range(eldiv):
        nodeTag += 1
        h = hbot + hbottap + (hmid/eldiv)*(j + 1)
        node(nodeTag, 0.0, 0.0, h)

    # Tapered middle section (Split into eldiv elements)
    for j in range(eldiv):
        nodeTag += 1
        h = hbot + hbottap + hmid + (hmidtap/eldiv)*(j + 1)
        node(nodeTag, 0.0, 0.0, h)

    # Top tower section (Split into eldiv elements)
    for j in range(eldiv):
        nodeTag += 1
        h = hbot + hbottap + hmid + hmidtap + (htop/eldiv)*(j + 1)
        node(nodeTag, 0.0, 0.0, h)

    # restraints
    # command: fix(nodeID, DOF1, DOF2, DOF3, DOF4, DOF5, DOF6) 0 = free, 1 = fixed
    # Use a fixed connection for now
    fix(1, 1, 1, 1, 1, 1, 1)

    # geometric transformation for beam-columns
    # command: geomTransf('Type', TransfTag)
    # see https://opensees.berkeley.edu/wiki/index.php/Linear_Transformation 
    geomTransf('PDelta', 1, 0, 1, 0) #columns

    # -------------------------------
    #       Define Elements
    # -------------------------------

    # initialize lists
    D = [] # diameters in
    A = [] # areas in^2
    I = [] # area moment of inertia in^4
    J = [] # polar moment of inertia in^4
    dM = [] # distributed masses kip*s^2/in
    V = [] # volumes in^3
    H = [] # height of each element in
    t = 0.21*inch # thickness is constant


    # Determining the diameters, areas, area moment of inertias, and polar moments of each discretized tower segment
    # diameters of tower
    dbot = 78.7*inch
    dmid = 62.9*inch
    dtop = 47.2*inch

    index = 0

    # Lower tower section (Split into eldiv elemenets)
    for j in range(eldiv):
        H.append(hbot/eldiv)
        D.append(dbot)
        A.append(((D[index])**2 - (D[index] - (2*t))**2)*math.pi/4)
        I.append(((D[index])**4 - (D[index] - 2*t)**4)*math.pi/64)
        J.append(((D[index])**4 - (D[index] - 2*t)**4)*math.pi/32)
        index += 1

    # Tapered lower section (Split into eldiv elements)
    for j in range(eldiv):
        H.append(hbottap/eldiv)
        d = ((j + 1)/(2*eldiv))*(dmid-dbot) + dbot # interpolating for the diameter based on eldiv
        D.append(d)
        A.append(((D[index])**2 - (D[index] - 2*t)**2)*math.pi/4)
        I.append(((D[index])**4 - (D[index] - 2*t)**4)*math.pi/64)
        J.append(((D[index])**4 - (D[index] - 2*t)**4)*math.pi/32)
        index += 1

    # Middle tower section (Split into eldiv elements)
    for j in range(eldiv):
        H.append(hmid/eldiv)
        D.append(dmid)
        A.append(((D[index])**2 - (D[index] - 2*t)**2)*math.pi/4)
        I.append(((D[index])**4 - (D[index] - 2*t)**4)*math.pi/64)
        J.append(((D[index])**4 - (D[index] - 2*t)**4)*math.pi/32)
        index += 1

    # Tapered middle section (Split into eldiv elements)
    for j in range(eldiv):
        H.append(hmidtap/eldiv)
        d = ((j + 1)/(2*eldiv))*(dtop-dmid) + dmid # interpolating for the diameter based on eldiv
        D.append(d)
        A.append(((D[index])**2 - (D[index] - 2*t)**2)*math.pi/4)
        I.append(((D[index])**4 - (D[index] - 2*t)**4)*math.pi/64)
        J.append(((D[index])**4 - (D[index] - 2*t)**4)*math.pi/32)
        index += 1

    # Top tower section
    for j in range(eldiv):
        H.append(htop/eldiv)
        D.append(dtop)
        A.append(((D[index])**2 - (D[index] - 2*t)**2)*math.pi/4)
        I.append(((D[index])**4 - (D[index] - 2*t)**4)*math.pi/64)
        J.append(((D[index])**4 - (D[index] - 2*t)**4)*math.pi/32)
        index += 1

    # Determining volumes of tower segments
    index = 0

    # Lower tower section (Split into eldiv elemenets)
    for j in range(eldiv):
        v = math.pi*(hbot/eldiv)*(D[index]**2 - (D[index]-2*t)**2)/4
        V.append(v)
        index += 1

    # Tapered lower section (Split into eldiv elements)
    for j in range(eldiv):
        dlower = ((dmid-dbot)*j)/(eldiv) + dbot
        dhigher = ((dmid-dbot)*(j+1))/(eldiv) + dbot
        v = math.pi*(hbottap/eldiv)*((dlower**2 +dlower*dhigher + dhigher**2)-((dlower-2*t)**2 + (dlower-2*t)*(dhigher-2*t) + (dhigher-2*t)**2))/12
        V.append(v)
        index += 1    

    # Middle tower section (Split into eldiv elements)
    for j in range(eldiv):
        v = math.pi*(hmid/eldiv)*(D[index]**2 - (D[index]-2*t)**2)/4
        V.append(v)
        index += 1

    # Tapered middle section (Split into eldiv elements)
    for j in range(eldiv):
        dlower = ((dtop-dmid)*j)/(eldiv) + dmid
        dhigher = ((dtop-dmid)*(j+1))/(eldiv) + dmid
        v = math.pi*(hmidtap/eldiv)*((dlower**2 +dlower*dhigher + dhigher**2)-((dlower-2*t)**2 +(dlower-2*t)*(dhigher-2*t) + (dhigher-2*t)**2))/12
        V.append(v)
        index += 1    

    # Top tower section (Split into eldiv elements)
    for j in range(eldiv):
        v = math.pi*(htop/eldiv)*(D[index]**2 - (D[index]-2*t)**2)/4
        V.append(v)
        index += 1


    mtotal = 14.101*kip/g
    vtotal = sum(V)
    htotal = hbot + hbottap + hmid + hmidtap + htop

    # Determining distributed masses of tower segments
    # Entire tower section (Split into 5*eldiv elemenets)
    for j in range(len(H)):
        distm = (mtotal*V[j])/(vtotal*H[j]) # mass/height constant
        dM.append(distm)

    # Defining the elements and determining areas, area moment of inertias, and polar inertias for each section
    # Cylindrical sections => Ix = Iy 
    # Cylindrical sections => J = Iz = 2*Ix = (D^4-(D-t)^4)*(pi/32) 

    # Assigning element properties
    # create elastic beam-column elements - 
    # command: element('elasticBeamColumn', eleTag, *eleNodes, Area, E_mod, G_mod, Jxx, Iy, Iz, transfTag, <'-mass', mass>, <'-cMass'>)
    # define the columns  
    for j in range(len(H)):
        element('elasticBeamColumn', j+1, j+1, j+2, A[j], Es, Gs, J[j], I[j], I[j], 1, '-mass', dM[j])

    # Uncomment to see how the turbine twists
    #node(nodeTag +1, 0.0, 10.0*ft, h)
    #geomTransf('Linear', 2, 0, 0, 1) #columns
    #element('elasticBeamColumn', len(H) + 1, len(H) + 1, len(H) + 2, A[len(H)-2], Es, Gs, J[len(H)-2], I[len(H)-2], I[len(H)-2], 2, '-mass', dM[len(H)-2])

    # assign additional masses
    # masses only act at nodes that have DoF
    # command: mass(nodeID, dx, dy, dz, r@x, r@y, r@z)
    # rotational inertia for a point mass: I = mr^2
    whub = 9.4*kip
    mhub = whub/g #kip*s^2/in
    ### to-do: the box is slightly higher than the last node of the tower, should we add another node? how would it be connected to the rest of the tower?
    hhub = 888*inch # height of hub
    #node(len(H) + 2, 0.0, 0.0, hhub) # CHANGE the top height it this is used
    #rigidLink('beam', len(H)+1, len(H)+2)
    mass(len(H) + 1, mhub, mhub, mhub, 0.0, 0.0, 0.0)

In [7]:

# units: in, kip, s
# dimensions
ft = 12.0
inch = 1.0
g = 386.1 #in/s^2
kip = 1.0
ksi = kip/(inch**2)
s = 1

# ----------------------------------
#       Define Sinusoidal Loading
# ----------------------------------

# Define parameters of the Sinusoidal Loading

T = 30 #Define how long the loading should go for
dt = 0.001 # Define the time step for input ground motion
tsteps = range(int((T/dt)))

# Define range of forcing frequency f (Hz)
f1 = 1.5/s
f2 = 20/s
df = 0.1 # Define discretization of forcing frequencies 
fsteps = range(int((f2-f1)/df))


t = [] # Define time list
for i in (tsteps):
    t.append(dt*i)

dfddug = pd.DataFrame(index=[t]) # Define dataframe with all ddug data
    
# -------------------------------
#       Set up Dynamic analysis
# -------------------------------

f = []
peakVx = []
peakVy = []
peakMx = []
peakMy = []
dfVx = pd.DataFrame(index=[t]) # Define dataframe with all Base Shear x data
dfVy = pd.DataFrame(index=[t]) # Define dataframe with all Base Shear y data
dfMx = pd.DataFrame(index=[t]) # Define dataframe with all Overturning Moment about the x axis data
dfMy = pd.DataFrame(index=[t]) # Define dataframe with all Overturning Moment about the y axis data

# # Define a function to return the maximum value of the steady state response
# def eleMax(items, start = 0, end=None):
#     return abs(max(items[start:end], key=abs))

buildmodel() # Build model to determine natural frequency
# --------------------------------
#       Perform eigen analysis
# --------------------------------

numEigen = 30
lameigenValues = eigen(numEigen)

wn = [i ** 0.5 for i in lameigenValues]
fn = [i / (2*math.pi) for i in wn]

print("Natural Frequencies:", fn, "Hz")

# compute the modal properties
#modalProperties("-print", "-file", "ModalReport.txt", "-unorm")

# -------------------------------------------
#       Plotting Structure and Modeshapes
# -------------------------------------------

# Display the active model with node and element tags
#plot_model("nodes","elements")

#plot_modeshape(1, 50)
#plot_modeshape(3, 50)
#plot_modeshape(6, 50)


# set the rayleigh damping factors for nodes & elements
rayleigh(0.0, 0.0, 0.0, 2*0.005/wn[0])

# Looping for each frequency
for i in fsteps:
    buildmodel() # Wipe and build new model for every frequency
    wipeAnalysis() # Wipe any previous pattern
    constraints('Plain')  			# how it handles boundary conditions
    numberer('Plain')
    system('BandGeneral')		    # how to store and solve the system of equations in the analysis
    algorithm('Newton')                 # use Newton-Raphson for linear analysis
    integrator('Newmark', 0.5, 0.25)        # create integration scheme, Newmark with alpha = 0.5, beta = 0.25
    analysis('Transient')      	    # define type of analysis static or transient

    # Create the convergence test, the norm of the residual with a tolerance of
    # 1e-12 and a max number of iterations of 10
    test('NormDispIncr', 1.0e-12,  10 )

# # # Run each ground motion per each frequency wf and extract the maximum moment per each sinusoidal ground motion
    f.append(f1 + (df*i)) # Keeping track of the frequencies
    Vx = [] # Resetting each vector for each frequency
    Vy = []
    Mx = []
    My = []
    setTime(0) # Reset time
    time = []
    
    # Setting up variables for the transient analysis
    tCurrent = getTime()
    tFinal = T
    ok = 0
    # Getting the correct ground motion per frequency step
    timeSeries('Trig', i+1, 0, T, 1/f[i], '-factor', g) 
    # patternTag, direction  accelTag
    pattern('UniformExcitation', i+1, 1, '-accel', i+1) # x direction

    # Perform the transient analysis
    while ok == 0 and tCurrent < tFinal:
        ok = analyze(1, dt)
        if ok != 0:
            print('WARNING' + str(tCurrent) + 'w' + str(i))
        # if the analysis fails try initial tangent iteration
            print("regular newton failed .. let's try an initial stiffness for this step")
            test('NormDispIncr', 1.0e-12,  100, 0)
            algorithm('ModifiedNewton', '-initial')
            ok = analyze(1, dt)
            if ok == 0:
                print("that worked .. back to regular newton")
            test('NormDispIncr', 1.0e-12,  10 )
            algorithm('Newton')
        # Store element forces per time step 
        tCurrent = getTime()
        time.append(tCurrent)     
        Vx.append(eleForce(1, 1)) # Vx
        Vy.append(eleForce(1, 2)) # Vy
        Mx.append(eleForce(1, 4)) # Mx
        My.append(eleForce(1, 5)) # My
    # Post-processing
    #maxVx = eleMax(Vx, int((T/dt)+1) - 20)
    #peakVx.append(maxVx)
    peakVx.append(abs(max(Vx, key=abs)))
    peakVy.append(abs(max(Vy, key=abs)))
    peakMx.append(abs(max(Mx, key=abs)))
    peakMy.append(abs(max(My, key=abs)))
    dfVx['w'+str(i)+"="+str(f[i])] = Vx
    dfVy['w'+str(i)+"="+str(f[i])] = Vy
    dfMx['w'+str(i)+"="+str(f[i])] = Mx
    dfMy['w'+str(i)+"="+str(f[i])] = My
    
dfVx.head(20)


Natural Frequencies: [1.7416607571903027, 1.7416607571903038, 12.931617421321723, 12.93161742132173, 31.41837575931187, 37.50532816108511, 37.50532816108545, 79.97732298602361, 79.97732298602384, 104.63755980264999, 133.1734638879487, 133.17346388795394, 196.81607870835393, 203.02975758995535, 203.0297575899589, 286.6503681199756, 286.6503681199774, 292.1269359117961, 381.99054626342144, 386.7368293026856, 386.73682930268916, 472.34995652879536, 503.8193891083675, 503.8193891083718, 568.2869993159859, 623.5401856468131, 623.540185646826, 648.9158433117034, 729.0586013039605, 766.585248992037] Hz


KeyboardInterrupt: 

In [ ]:


#plt.plot(time, dfVx['w1'])
# dfMy.head(30)

#fig, ax = plt.subplots(dpi = 200, figsize=(20,3)) #time
fig, ax = plt.subplots(dpi = 150, figsize=(20,2)) #freq


# plt.plot(time, dfVx.iloc[:,50])
# plt.ylabel('Base Shear (kip)')
# plt.xlabel('Time (s)')
# plt.title('Base Shear vs Time')
# plt.grid(b=True)
# plt.show()

# ax.plot(time, dfMy.iloc[:,0]/12)
# ax.set_ylabel('Overturning Moment (kip*ft)')
# ax.set_xlabel('Time (s)')
# ax.set_title('Overturning Moment vs Time')
# ax.grid(True)
# plt.tight_layout()
# plt.xlim([0,T])

ax.plot(f, peakVx)
ax.set_ylabel('Peak Base Shear (kip)')
ax.set_xlabel('Forcing Frequency (Hz)')
ax.set_title('Peak Base Shear vs Forcing Frequency')
ax.grid(True)
plt.tight_layout()
plt.xlim([0,f2])


# plt.plot(f, peakVx)
# plt.ylabel('Peak Base Shear (kip)')
# plt.xlabel('Frequency (Hz)');


In [5]:


#print(f)
print(peakVx)
#print(peakMy)
#print(f2)




[19.002342822249087, 19.566940936460107, 20.1138490683713, 20.748209664102507, 21.58715114034604, 22.374796039360096, 23.214131356280195, 24.188471466051105, 25.35344316262986, 26.661841040255094, 28.028508312320174, 29.74625003463156, 31.680626177444985, 33.98123716815003, 36.894589039047816, 40.19005540362319, 44.21700273798738, 50.23523343518887, 58.06475356706478, 68.19839063646242, 86.19299308032858, 116.76260145890231, 185.833738934913, 491.4781927189702, 671.8943774745247, 192.9705877613047, 110.59665484314736, 76.31026454264499, 57.688159497926684, 45.958607928001896, 37.87385369384337, 31.936740427081396, 27.388559215055043, 23.847061544744506, 20.99646465343776, 18.592031632591812, 16.497026945843892, 14.982713572641233, 13.508011016166668, 12.282512442048791, 11.285758480161538, 11.177724960237493, 11.196681449384782, 11.146791797028321, 11.18944296223295, 11.25129458337217, 11.23027735932596, 11.102782312264061, 11.258018044512154, 11.267020119345306, 11.335258664547881, 11

In [ ]:
# 3-D Plotting

#https://stackoverflow.com/questions/51765184/how-to-3d-plot-function-of-2-variables-in-python?noredirect=1&lq=1
#https://stackoverflow.com/questions/45396059/multiple-independent-lines-in-the-same-3d-axes
fig = plt.figure()
ax = fig.gca(projection='3d')

freq = pd.DataFrame(index=[t])

for i in range(20): # all frequencies: len(f)
    freq[i] =  [f[i]] * len(time)
    ax.plot(time,freq[i], dfVx.iloc[:,i])
    

# make labels
ax.set_xlabel('Time (s)')
ax.set_ylabel('Forcing Frequency (Hz)')
ax.set_zlabel('Base Shear (kip)')

plt.show()

In [ ]:
# 3-D Plotting Surface

#https://stackoverflow.com/questions/12423601/simplest-way-to-plot-3d-surface-given-3d-points
from matplotlib.ticker import MaxNLocator
from matplotlib import cm
import numpy
from numpy.random import randn
from scipy import array, newaxis

Vx3dall = []
Vx3d = []

for i in range(200): #len(f)
    for j in range(20000): #len(time)
        Vx3d = [time[j], f[i], dfVx.iat[j,i]]
        Vx3dall.append(Vx3d)

DATA = array(Vx3dall)
Xs = DATA[:,0]
Ys = DATA[:,1]
Zs = DATA[:,2]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_trisurf(Xs, Ys, Zs, cmap=cm.jet, linewidth=0)
fig.colorbar(surf)

ax.xaxis.set_major_locator(MaxNLocator(5))
ax.yaxis.set_major_locator(MaxNLocator(6))
ax.zaxis.set_major_locator(MaxNLocator(5))

# make labels
ax.set_xlabel('Time (s)')
ax.set_ylabel('Forcing Frequency (Hz)')
ax.set_zlabel('Base Shear (kip)')

fig.tight_layout()

plt.show()
